## Asyncronous computing

This notebooks shows how to create tasks and use them.

In [61]:
versioninfo()

Julia Version 0.6.0-rc1.0
Commit 6bdb3950bd (2017-05-07 00:00 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-4650U CPU @ 1.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


In [6]:
v = rand(1:3,100000);
v2 = rand(1:3,100000);

In [108]:
type LinearModel
    W::Vector
end

function predict(h::LinearModel, x::Vector)
    return sign(h.W'*x)
end

predict (generic function with 2 methods)

In [64]:
using MLDatasets

In [69]:
expand( :(h.W'*x))

:(Ac_mul_B((Core.getfield)(h, :W), x))

In [39]:
X,y = MLDatasets.MNIST.traindata();

In [40]:
size(X)

(28, 28, 60000)

In [157]:
X = reshape(X,(784,60000));
h = LinearModel(rand(784));
y_hat = zeros(size(X,2));

In [184]:
@time begin
    for m in size(X,2) 
          y_hat[m] = predict(h, X[:,m])
    end
end

  0.000046 seconds (8 allocations: 6.375 KiB)


In [182]:
@time begin
    @sync for m in size(X,2) 
          y_hat[m] = predict(h, X[:,m])
    end
end

  0.000059 seconds (12 allocations: 6.563 KiB)


In [174]:
@time begin
    for i in 1:20
      @async for m in size(X,2) 
          y_hat[m] = predict(h, view(X,:,m))
      end
    end
end

  0.000093 seconds (39 allocations: 752 bytes)


## More examples

In [82]:
a2() = det(rand(10000, 10000))

a2 (generic function with 1 method)

In [ ]:
b = Task(a2);

In [ ]:
typeof(b)

In [ ]:
istaskdone(b)

In [ ]:
schedule(b);
wait()

In [ ]:
istaskdone(b)

In [ ]:
yield();

In [ ]:
?schedule